In [1]:
import numpy as np
import pandas as pd
import re

Импортируется csv-файл, из которого убраны

In [129]:
realty = pd.read_csv('realty_very_clean.csv', encoding='utf-8')

In [131]:
realty = realty.rename(columns={'Unnamed: 0': 'id'})

In [162]:
realty[:15]

,id,request,type,amount
0,0,youпитер,Квартиры,3
1,1,yes,Квартиры,10
2,2,yamaha,Квартиры,3
3,3,xtk.crbywtd,Квартиры,3
4,4,xiaomi,Квартиры,3
5,5,xbox 360,Квартиры,4
6,6,wifi,Квартиры,2
7,7,wi-fi,Квартиры,13
8,8,wi-fi,"Дома, дачи, коттеджи",2
9,9,wellton park,Квартиры,2


Первый вид списков для парсера — варианты формулировок для количества комнат в квартире. 
Остается проблема, из-за которой пасрер может ловить не все: в функции каждый запрос разбивается по пробелу и сравнивается со словарем, соответственно, объекты типа "1 ком" не ловятся, так как с таким образцом сравниваются отдельно "1" и отдельно "ком". Возможное решение — сравнивать весь запрос целиком, в таком случае будут возникать ошибки, если запрос выглядит как "сдать однокомнатную квартиру".

In [100]:
one_room = ['1 ком', '1 квартиру', '1 квартира', '1-комнатную',  '1-к', '1к квартира', '1 комнатную', '1-комнатная',
            '1-но', '1к квартиру', '1к.кв', '1 комн', '1-х', '1 комн.', '1-ком', '1к', '1 комнатная', '1 комната', '1кв',
            '1ккв', '1ком', '1комн', '1комнатная', '1комнатную', '1х', 'малосемейка', 'однокомнатная', 'однокомнатную',
            'однушка', 'однушку']

In [101]:
two_rooms = ['2-3', '2 комнатную', '2-комнатная', '2-х', 'двушки', '2 к кв', '2 к', '2-к', 'двухкомнатные', '2 ккв', '2 ком',
             '2ком', '2 -комнатные', '2комнатная', '2к', '2 -хкомнатные', 'евродвушка', '2 комн', '2 комн кв', '2 комн.',
             '2 комнатная', '2 комнатные', '2 комнаты', '2 х', '2-комн', '2комн', '2х', '2хкомн', '2хкомнатная', 'двухкомнатная'
             'двухкомнатную', 'двушка', 'двушку']

In [102]:
three_rooms = ['3-к квартира', 'трехкомнатные', 'трехкомнатную', '3комнатные', '3 -х комнатные', '3к кв', '3к', '3 комнатные', 
               '3 к', '3комнатная', '3 комнаты', '3 х комнатная', '3комнаятная', '3х', 'трехкомнатная', '3-комн', '3ккв', 
               '3 комнат', '3ккв', '3-комнатная', '3-х', '3 комнатная', '3  квартира', '3 -комнатные', '3 комн', '3 комнатную',
               '3 х комнатные', '3-к']

In [104]:
four_rooms = ['четырехкомнатные', '4 комнатная', '4 к квартира', '4-к', '4-комнатная', '4-х', 'четырехкомнатных', 
             'четырехкомнатная', 'четырёхкомнатные']

In [217]:
studio = ['студия', 'гостинка', 'гостинку', 'студию', 'cтудия', 'cтудию']

Списки для видов собственности, локации и остальных особенностей.

In [163]:
own_property = ['продам', 'обмен', '1/2', 'продажа', 'продаю', 'куплю', 'купить', 'переуступка', 'вторичка', 'меняю', 'в рассрочку', 'продам']

In [227]:
rent = ['сдам', 'снять', 'сниму', 'аренда', 'хозяина', 'сдаю', 'посуточно', 'по суточно', 
        'длительный срок', 'с арендаторами' , 'хозяйкой', 'можно с животными',
        'подселение', 'общежитие квартирного типа', 'комната в общежитие', 'сдача', 'общежитие', 'на месяц', 'почасовая', 'cдаю',
        'cнять', 'cниму', 'cдам', 'cдача']

In [166]:
center_location = ['центр', 'в центре города']

In [167]:
sea_location = ['у моря', 'с видом на море', 'у моря', 'на берегу', 'на берегу моря','рядом с морем', 'дом на берегу черного моря', 'курортная зона', 'первая линия']

In [171]:
warm = ['индивидуальное', 'автономное']

In [172]:
design = ['ремонт', 'ремонтом', 'отделкой', 'евроремонт', 'мебелью', 'дизайнерский']

Простой парсер, построенный по следующему принципу: в случае совпадение элемента запроса с примером из словаря, он добавляет соответствующую пометку. На выходе получается датафрейм с отметкой для каждой категории.

In [228]:
def searcher(request):
    n_property = []
    rooms = []
    location = []
    center_location = []
    design = []
    warm = []
        
    for word in request.split():
        if word in rent:
                n_property.append('rent')
        if word in own_property:
                n_property.append('own_property')
        if word in one_room:
                rooms.append('one_room')
        if word in two_rooms:
                rooms.append('two_rooms')
        if word in three_rooms:
                rooms.append('three_rooms')
        if word in four_rooms:
                rooms.append('four_rooms')          
        if word in studio:
                rooms.append('studio')
        if word in warm:
                warm.append('warm')
        if word in sea_location:
            location.append('sea_location')        
        if word in center_location:
                location.append('center_location')
        if word in design:
                design.append('design')                

       
    return request, n_property, rooms, location, warm, design 




Вариант парсера, ищущего только по комнатам.

In [170]:
#def room_searcher(request):
#    rooms = []
    
#    for word in request.split():
#        if word in one_room:
#                rooms.append('one_room')
#        if word in two_rooms:
#                rooms.append('two_rooms')
#        if word in three_rooms:
#                rooms.append('three_rooms')
#        if word in four_rooms:
#                rooms.append('four_rooms')        
#        if word in studio:
#                rooms.append('studio')                          
#    return request, rooms


In [229]:
searcher('1к квартиры посуточно')

('1к квартиры посуточно', ['rent'], ['one_room'], [], [], [])

Парсер проходит по списку запросов:

In [230]:
def great_room_seacher (real_requests):
    return [room_searcher(request) for request in real_requests]
        

In [231]:
super_marked = great_seacher(realty['request'])

In [232]:
type(super_marked)

list

In [233]:
super_marked

[('youпитер', [], [], [], [], []),
 ('yes', [], [], [], [], []),
 ('yamaha', [], [], [], [], []),
 ('xtk.crbywtd', [], [], [], [], []),
 ('xiaomi', [], [], [], [], []),
 ('xbox 360', [], [], [], [], []),
 ('wifi', [], [], [], [], []),
 ('wi-fi', [], [], [], [], []),
 ('wi-fi', [], [], [], [], []),
 ('wellton park', [], [], [], [], []),
 ('wellton', [], [], [], [], []),
 ('vjht', [], [], [], [], []),
 ('vjht', [], [], [], [], []),
 ('viva', [], [], [], [], []),
 ('vip', [], [], [], [], []),
 ('vip', [], [], [], [], []),
 ('vbhf', [], [], [], [], []),
 ('vbhf', [], [], [], [], []),
 ('valo', [], [], [], [], []),
 ('v', [], [], [], [], []),
 ('up светлановский', [], [], [], [], []),
 ('up комендантский', [], [], [], [], []),
 ('ultra city', [], [], [], [], []),
 ('ufp', [], [], [], [], []),
 ('tribeca', [], [], [], [], []),
 ('technics', [], [], [], [], []),
 ('sunrise city', [], [], [], [], []),
 ('sunrise', [], [], [], [], []),
 ('studia', [], [], [], [], []),
 ('spar', [], [], [], [], 

Список превращаем в датафрейм:

In [234]:
labels = ['request', 'property', 'rooms', 'location', 'warm', 'design'] 


In [277]:
marked_frame = pd.DataFrame.from_records(super_marked, columns=labels)

Вот на этом этапа обнаружилось интересное: некоторые запросы, которые вроде должны ловиться (тот же "сдам") не выделялись. Подсказкой стало то, что запросы отсортированы более-менее по алфавиту: в некоторых таких запросах пользователи вводили латинскую с, а не русскую! (facepalm) Списки в словарях пришлось удаваивать для таких случаев.

In [278]:
marked_frame[160:190]

,request,property,rooms,location,warm,design
160,ecoville,[],[],[],[],[]
161,eco life,[],[],[],[],[]
162,e,[],[],[],[],[]
163,dreams,[],[],[],[],[]
164,dream town,[],[],[],[],[]
165,docklands,[],[],[],[],[]
166,ddr3,[],[],[],[],[]
167,d,[],[],[],[],[]
168,d,[],[],[],[],[]
169,d,[],[],[],[],[]


In [237]:
#marked_frame2 = pd.DataFrame.from_records(super_marked, columns=labels)

От начального этапа остались квадратные скобки — убираем их.

In [279]:
marked_frame['rooms'] = marked_frame['rooms'].str.get(0)
marked_frame['property'] = marked_frame['property'].str.get(0)
marked_frame['location'] = marked_frame['location'].str.get(0)
marked_frame['warm'] = marked_frame['warm'].str.get(0)
marked_frame['design'] = marked_frame['design'].str.get(0)

In [280]:
marked_frame[2250:2270]

,request,property,rooms,location,warm,design
2250,щекино,NaN,NaN,NaN,NaN,NaN
2251,щекино,NaN,NaN,NaN,NaN,NaN
2252,щекино,NaN,NaN,NaN,NaN,NaN
2253,щекавцево,NaN,NaN,NaN,NaN,NaN
2254,щедрино парковая,NaN,NaN,NaN,NaN,NaN
2255,щедрино,NaN,NaN,NaN,NaN,NaN
2256,щедрино,NaN,NaN,NaN,NaN,NaN
2257,щедрина,NaN,NaN,NaN,NaN,NaN
2258,щедрина,NaN,NaN,NaN,NaN,NaN
2259,щедрина,NaN,NaN,NaN,NaN,NaN


In [238]:
#это на случай, если бы мы хотели выкинуть колонки, где совсем ничего нет
#marked_frame = marked_frame.dropna(subset=['rooms'])

Теперь исправляем NaN на более подходящие нам значения

In [281]:
marked_frame['rooms'].fillna('no_rooms', inplace=True)
marked_frame['property'].fillna('no_property', inplace=True)
marked_frame['location'].fillna('no_locations', inplace=True)
marked_frame['warm'].fillna('no_warm', inplace=True)
marked_frame['design'].fillna('no_design', inplace=True)

In [282]:
marked_frame.sort_values(by=['rooms'])

,request,property,rooms,location,warm,design
85639,4-х связистов,no_property,four_rooms,no_locations,no_warm,no_design
85640,4-х комнатные,no_property,four_rooms,no_locations,no_warm,no_design
4416,четырёхкомнатные,no_property,four_rooms,no_locations,no_warm,no_design
4417,четырехкомнатные,no_property,four_rooms,no_locations,no_warm,no_design
4418,четырехкомнатная квартира,no_property,four_rooms,no_locations,no_warm,no_design
4419,четырехкомнатная,no_property,four_rooms,no_locations,no_warm,no_design
85641,4-х комнатная квартира,no_property,four_rooms,no_locations,no_warm,no_design
85642,4-х комнатная,no_property,four_rooms,no_locations,no_warm,no_design
85643,4-комнатная,no_property,four_rooms,no_locations,no_warm,no_design
29278,продажа четырехкомнатных квартир,own_property,four_rooms,no_locations,no_warm,no_design


In [249]:
marked_frame.to_csv('marked_rooms.csv', encoding = 'utf-8')

## Машинное обучение

В данной задаче для каждого из вида признаков на текущем этапе придется обучать алгоритм отдельно, так как между ними нет никакой взаимосвязи. В результате нескольких опытов наилучший результат показал метод ближайших соседей.

In [250]:
from sklearn.model_selection import train_test_split

#### 1. Комнаты

In [262]:
marked_rooms = marked_frame.drop(columns=['property', 'location', 'warm', 'design'])

In [263]:
train, test = train_test_split(marked_rooms, test_size=0.3)

In [264]:
y_train = train['rooms'] 
y_test = test['rooms'] 

In [266]:
train[:10]

,request,rooms
52843,краснооктябрьский район 2комнатная,two_rooms
11091,ул серова,no_rooms
7661,федеральная,no_rooms
14059,тлюстенхабль,no_rooms
66280,дорогино,no_rooms
67880,детский,no_rooms
75352,верхнеполевая,no_rooms
6916,фучика 145,no_rooms
66743,дом из бруса,no_rooms
9354,улица велинградская,no_rooms


In [267]:
def set_clean(s):
    try:
        clean_line = re.sub('[\W\d_-]+', ' ', s.lower().strip())
        return re.sub(' +', ' ', clean_line)
    except AttributeError:
        ''#print ("this was a series")

In [268]:
import re
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [269]:
train_clean = train.applymap(set_clean)
test_clean = test.applymap(set_clean)

In [270]:
len(train)

61513

In [271]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
def tfidf_vec(voc=None):
    if(voc):
        vectorizer = TfidfVectorizer(ngram_range=(1, 3),
                                     min_df=5,   
                                     max_df=0.9)  
        tr = vectorizer.fit_transform(train["request"]) 
        te = vectorizer.fit_transform(test["request"]) 
        return (tr, te)
    else:
        vectorizer = TfidfVectorizer() 
        tr = vectorizer.fit_transform(train["request"]) 
        voc = vectorizer.get_feature_names()
        vectorizer = CountVectorizer(vocabulary=voc) 
        te = vectorizer.fit_transform(test["request"]) 
        return (tr, te)
train_counts, test_counts = tfidf_vec() 

tfidf_transformer = TfidfTransformer()

x_train = tfidf_transformer.fit_transform(train_counts)
x_test = tfidf_transformer.fit_transform(test_counts)

In [274]:
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [276]:
model = KNeighborsClassifier()
model.fit(x_train, y_train)
print(model)

expected = y_test
predicted = model.predict(x_test)

print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')


C:\Users\ksenia\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

 four_rooms       0.00      0.00      0.00         4
   no_rooms       1.00      1.00      1.00     26265
   one_room       1.00      0.19      0.31        27
     studio       1.00      0.64      0.78        22
three_rooms       1.00      0.06      0.11        17
  two_rooms       1.00      0.21      0.34        29

avg / total       1.00      1.00      1.00     26364

[[    0     4     0     0     0     0]
 [    0 26265     0     0     0     0]
 [    0    22     5     0     0     0]
 [    0     8     0    14     0     0]
 [    0    16     0     0     1     0]
 [    0    23     0     0     0     6]]


#### 2. Владение

In [283]:
marked_property = marked_frame.drop(columns=['rooms', 'location', 'warm', 'design'])

In [284]:
train, test = train_test_split(marked_property, test_size=0.3)

In [285]:
y_train = train['property'] 
y_test = test['property'] 

In [286]:
train_clean = train.applymap(set_clean)
test_clean = test.applymap(set_clean)

In [287]:
def tfidf_vec(voc=None):
    if(voc):
        vectorizer = TfidfVectorizer(ngram_range=(1, 3),
                                     min_df=5,   
                                     max_df=0.9)  
        tr = vectorizer.fit_transform(train["request"]) 
        te = vectorizer.fit_transform(test["request"]) 
        return (tr, te)
    else:
        vectorizer = TfidfVectorizer() 
        tr = vectorizer.fit_transform(train["request"]) 
        voc = vectorizer.get_feature_names()
        vectorizer = CountVectorizer(vocabulary=voc) 
        te = vectorizer.fit_transform(test["request"]) 
        return (tr, te)
train_counts, test_counts = tfidf_vec() 

tfidf_transformer = TfidfTransformer()

x_train = tfidf_transformer.fit_transform(train_counts)
x_test = tfidf_transformer.fit_transform(test_counts)

In [288]:
model = KNeighborsClassifier()
model.fit(x_train, y_train)
print(model)

expected = y_test
predicted = model.predict(x_test)

print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
              precision    recall  f1-score   support

 no_property       0.99      1.00      1.00     25798
own_property       0.83      0.73      0.77       300
        rent       0.94      0.64      0.76       266

 avg / total       0.99      0.99      0.99     26364

[[25748    41     9]
 [   81   218     1]
 [   91     5   170]]
